## **Notebook Description**

**Function**

Uses the trained model to make predictions on a new, unseen dataset. 

**Output**

Csv with predictions on unseen dataset. 

## **Import Libraries**

In [1]:
import numpy as np
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.data import Data

## **Input**

In [2]:
# Paste model path
model_iteration = 'model'

In [3]:
# Input Pytorch Object Date
# Copy paste in here
object = 'predict_1'

In [4]:
# Set the slicing range for the indices (labeled nodes)
sensor_indices = slice(0, 592) 

In [5]:
#Path to save script
file_path = rf"data\2_prediction_testing\sensor_predictions.csv"

## **Classes**

**Load the Trained Model**

In [6]:
# Note to self, convert this class to a .py script later on

class SimpleGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)  # First GCN layer
        self.conv2 = GCNConv(16, num_classes)        # Second GCN layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return x  

## **Loading & Settings**

In [7]:
num_node_features = 4  
num_classes = 1  

model = SimpleGCN(num_node_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  

In [8]:
# Load the saved state_dict
model.load_state_dict(torch.load(rf'data\1_pytorch_model\{model_iteration}.pth'))

<All keys matched successfully>

In [9]:
# Set the model to evaluation mode
model.eval()

SimpleGCN(
  (conv1): GCNConv(4, 16)
  (conv2): GCNConv(16, 1)
)

In [10]:
# Load your data object for prediction
new_data = torch.load(rf'data\torch_data_object_prediction\{object}.pt')

print('loaded')

loaded


In [11]:
# If 'new_data' is a single Data object, wrap it in a list for DataLoader compatibility
if isinstance(new_data, Data):
    new_data = [new_data]

In [12]:
# Create a DataLoader for your new dataset
data_loader = DataLoader(new_data, batch_size=1, shuffle=False)

## **Predictions**

In [13]:
# Make predictions
predictions = []
for data in data_loader:
    with torch.no_grad():
        output = model(data)
        sensor_output = output[sensor_indices]  # Adjusted slicing
        predictions.append(sensor_output.squeeze(-1).detach().numpy())

In [14]:
predictions

[array([412.00732, 416.09308, 420.17877, 424.26453, 428.35022, 432.43597,
        436.52173, 440.60745, 444.69318, 448.77893, 452.86462, 456.95035,
        461.0361 , 465.12183, 411.83395, 415.64728, 419.46063, 423.274  ,
        427.08734, 430.9007 , 434.71408, 438.5274 , 442.34076, 446.15408,
        449.96744, 453.7808 , 457.59415, 461.4075 , 465.22086, 413.71497,
        417.52832, 421.34167, 425.15506, 428.96838, 432.7817 , 436.5951 ,
        440.40845, 444.2218 , 448.03516, 451.84845, 455.6618 , 459.4752 ,
        463.2885 , 467.10187, 414.70966, 418.52304, 422.33636, 426.14975,
        429.96307, 433.7764 , 437.58972, 441.4031 , 445.2165 , 449.02982,
        452.8432 , 456.65652, 460.4699 , 464.28323, 468.09656, 415.92377,
        419.73712, 423.55048, 427.36383, 431.1772 , 434.99054, 438.8039 ,
        442.61725, 446.4306 , 450.24396, 454.05728, 457.87064, 461.68396,
        465.4973 , 469.31067, 417.4298 , 421.2431 , 425.05646, 428.8698 ,
        432.6832 , 436.49652, 440.3098

In [15]:
# Display the shape of predictions
predictions_array = np.array(predictions)
print("Shape of predictions array:", predictions_array.shape)

Shape of predictions array: (1, 592)


In [16]:
print("Output shape:", output.shape)
print("Sensor output shape:", sensor_output.shape)

Output shape: torch.Size([896, 1])
Sensor output shape: torch.Size([592, 1])


## **Export to csv**

In [17]:
# Reshape predictions_array to have shape (592, 1)
predictions_array = np.vstack(predictions)
predictions_array = predictions_array.T
print("Shape of sensor predictions array:", predictions_array.shape)

Shape of sensor predictions array: (592, 1)


In [18]:
# Export rounded predictions to CSV in the same folder as the script
np.savetxt(file_path, predictions_array, delimiter=",")
print(f"Saved predictions to: {file_path}")

Saved predictions to: data\2_prediction_testing\sensor_predictions.csv
